In [6]:
import requests
import csv

In [76]:
response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=NFL&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3")

data = response.json() 
hits = data["response"]["meta"]["hits"]
max_page = int(hits / 10 - 1)

In [51]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news) 

In [94]:
web_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=NFL&begin_date=20190101&api-key=Nw4HZ3XGu0pCTuVFupaz1IQgzko2lyZ3"
data_list = []
for i in range(0,max_page):
    reques_url = web_url + "&page=" + str(i)
    response = requests.get(reques_url)
    data = response.json() 
    new = parse_articles(data)
    data_list = data_list+new


keys = data_list[0].keys()
with open('ny_nfl_19Mar.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_list)    
